In [1]:
import pandas as pd
import os
df=pd.read_excel("Data_For Model 20230107.xlsx")

In [ ]:
from tqdm import tqdm
from tensorflow.keras.layers import *
from tensorflow import keras
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import StratifiedKFold
from keras.utils.np_utils import to_categorical
from sklearn.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
tqdm.pandas()
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" ###指定此处为-1即可

In [2]:
df["Label"].value_counts()

Label
IS    8131
ES    3093
iS       1
b        1
Name: count, dtype: int64

In [3]:
tmpd={"IS":0,"ES":1}

In [4]:
df["label"]=df["Label"].map(tmpd)

In [5]:
df=df.dropna(subset=["label"])

In [6]:
df["label"].value_counts()

label
0.0    8131
1.0    3093
Name: count, dtype: int64

In [7]:
df["text"]=df["Text"]

In [8]:
df

,Unnamed: 0,Text,Label,label,text
0,NaN,别担心，这种情况是可以解决的,ES,1.0,别担心，这种情况是可以解决的
1,NaN,遇到这种事情是很麻烦的，不是三言两语可以解决的，要具体问题具体对待，希望你们和睦相处，开心快乐！,ES,1.0,遇到这种事情是很麻烦的，不是三言两语可以解决的，要具体问题具体对待，希望你们和睦相处，开心快乐！
2,NaN,我的已经到了，希望大家的也马上到达。,ES,1.0,我的已经到了，希望大家的也马上到达。
3,NaN,呵呵，人力搜索中——查找！,IS,0.0,呵呵，人力搜索中——查找！
4,NaN,寻找共同爱好，比如游戏等,IS,0.0,寻找共同爱好，比如游戏等
...,...,...,...,...,...
11221,NaN,这个也很好玩耶！,ES,1.0,这个也很好玩耶！
11222,NaN,太强了,ES,1.0,太强了
11223,NaN,我非常赞同这位同学的看法,ES,1.0,我非常赞同这位同学的看法
11224,NaN,我觉得您的想法和思维导图做的不错！,ES,1.0,我觉得您的想法和思维导图做的不错！


#### 去除停用词

In [9]:
import re
import jieba_fast as jieba
ting=[i.strip() for i in open("stop_words_ch-停用词表.txt",encoding='gbk').readlines()]
def fenci(text):
    return [i for i in jieba.cut(str(text)) if i not in ting] 

df["text"]=df["text"].apply(lambda x:fenci(x))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\jhg\AppData\Local\Temp\jieba.cache
Loading model cost 0.882 seconds.
Prefix dict has been built succesfully.


In [10]:
df["text"]=df["text"].apply(lambda x:" ".join(x))

### 深度学习模型

#### w2v加载

In [ ]:
import pandas as pd
import gensim
if not os.path.exists("w2v"):
    w2v_model = gensim.models.Word2Vec(list(df["text"]), vector_size=128, epochs=10, min_count=0)
    word_vectors = w2v_model.wv
    w2v_model.save("w2v")
else:
    print ("直接加载训练好的w2v模型")
    w2v_model=gensim.models.Word2Vec.load("w2v")
    word_vectors = w2v_model.wv
    print ("w2v模型加载完毕")

#### 转向量

In [ ]:
import joblib
x_train=list(df["text"])
if not os.path.exists("tokenizer.joblib"):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x_train)  #统计每个词对应的数字，以便于将文本转化成向量
    joblib.dump(tokenizer,"tokenizer.joblib")
else:
    print("加载token")
    tokenizer=joblib.load("tokenizer.joblib")
train_sequence = tokenizer.texts_to_sequences(x_train)#将所有的文本转化成向量
MAX_SEQUENCE_LENGTH=64 #最大长度
EMBEDDING_DIM = 128 #向量维度

y_train =df["label"]
y_train = to_categorical(y_train)  #将标签 one-hot
y_train = y_train.astype(np.int32)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
train_pad = pad_sequences(train_sequence, maxlen=MAX_SEQUENCE_LENGTH) #将每条文本按照最大长度补0

#### 嵌入矩阵

In [ ]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM), dtype=np.float32)
not_in_model = 0
in_model = 0
embedding_max_value = 0
embedding_min_value = 1
not_words = []

for word, i in tqdm(word_index.items()):
    if word in w2v_model.wv.key_to_index:
        in_model += 1
        embedding_matrix[i] = np.array(w2v_model.wv[word])
        embedding_max_value = max(np.max(embedding_matrix[i]), embedding_max_value)
        embedding_min_value = min(np.min(embedding_matrix[i]), embedding_min_value)
    else:
        not_in_model += 1
        not_words.append(word)

#### 构建二分类模型

In [ ]:
def get_cnnmodel(class_num=2):
    embed = Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH,
				  trainable=True)  #定义一个词嵌入层,将句子转化成对应的向量
    inputs_sentence = Input(shape=(MAX_SEQUENCE_LENGTH,))#设置输入向量维度
    sentence =embed(inputs_sentence)#定义词嵌入层
    kernel_sizes=[13,14,15]
    convs = []
    max_poolings = []
    for kernel_size in kernel_sizes:
        convs.append(Conv1D(64, kernel_size, activation='relu'))
        max_poolings.append(GlobalMaxPooling1D())
    convs1 = []
    for i in range(len(kernel_sizes)):
        c = convs[i](sentence)
        c = max_poolings[i](c)
        convs1.append(c)
    x = Concatenate()(convs1)
#     x=Dense(64, activation='relu')(x)
    dp=Dropout(0.1)(x)
    output = Dense(class_num, activation='softmax')(dp)#softmax层
    model = Model(inputs=[inputs_sentence], outputs=output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])#定义损失函数，优化器，评分标准
    model.summary()
    return model

### 交叉验证

In [13]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict
from sklearn.naive_bayes import GaussianNB
kf = KFold(n_splits=5, shuffle=True, random_state=42)

### 每一折的交叉验证报告 手动平均即为交叉验证的平均结果

In [14]:
X=train_pad
y=y_train
true_labels_all = []
predicted_labels_all = []
for train_index, test_index in kf.split(X,y):

    # 划分训练集和测试集
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    
    model=get_cnnmodel()
    callbacks = [EarlyStopping(monitor='val_acc', min_delta=0.001, patience=10),
			 ModelCheckpoint("textcnn.hdf5", monitor='val_acc',
							 mode='max', verbose=0, save_best_only=True,save_weights_only=True)]   
    #设置模型提前停止,停止的条件是验证集val_acc两轮已经不增加,保存验证集val_acc最大的那个模型,名称为new_cnn.hdf5
    history=model.fit(X_train,y_train, batch_size=64, epochs=20, callbacks=callbacks,validation_data=(X_test,y_test))

    model.load_weights("textcnn.hdf5")
    testpre=np.argmax(model.predict([X_test]),axis=1)
    import matplotlib.pyplot as plt
    val_loss = history.history['val_loss']
    loss = history.history['loss']
    epochs = range(1, len(loss ) + 1)
    plt.title('cnn_Loss')
    plt.plot(epochs, loss, 'red', label='Training loss')
    plt.plot(epochs, val_loss, 'blue', label='Validation loss')
    plt.legend()
    plt.show()
    plt.cla()

    val_loss = history.history['val_acc']
    loss = history.history['acc']
    epochs = range(1, len(loss ) + 1)
    plt.title('cnn_acc')
    plt.plot(epochs, loss, 'red', label='Training acc')
    plt.plot(epochs, val_loss, 'blue', label='Validation acc')
    plt.legend()
    plt.show()
    from sklearn.metrics import classification_report
    tmpd={"IS":0,"ES":1}
    print (classification_report(np.argmax(y_test,axis=1),testpre,digits=4,target_names=list(tmpd.keys())))
    true_labels_all.extend(np.argmax(y_test,axis=1))
    predicted_labels_all.extend(testpre)

D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classification report for fold:
              precision    recall  f1-score   support

          IS     0.9468    0.7461    0.8345      1646
          ES     0.5591    0.8848    0.6852       599

    accuracy                         0.7831      2245
   macro avg     0.7529    0.8154    0.7599      2245
weighted avg     0.8433    0.7831    0.7947      2245




D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classification report for fold:
              precision    recall  f1-score   support

          IS     0.9335    0.7392    0.8250      1614
          ES     0.5646    0.8653    0.6834       631

    accuracy                         0.7746      2245
   macro avg     0.7491    0.8022    0.7542      2245
weighted avg     0.8298    0.7746    0.7852      2245




D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classification report for fold:
              precision    recall  f1-score   support

          IS     0.9425    0.7407    0.8295      1639
          ES     0.5559    0.8779    0.6807       606

    accuracy                         0.7777      2245
   macro avg     0.7492    0.8093    0.7551      2245
weighted avg     0.8382    0.7777    0.7894      2245




D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classification report for fold:
              precision    recall  f1-score   support

          IS     0.9456    0.7253    0.8210      1631
          ES     0.5493    0.8893    0.6791       614

    accuracy                         0.7702      2245
   macro avg     0.7475    0.8073    0.7500      2245
weighted avg     0.8372    0.7702    0.7822      2245




D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
D:\anaconda\root\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Classification report for fold:
              precision    recall  f1-score   support

          IS     0.9367    0.7214    0.8151      1601
          ES     0.5589    0.8787    0.6832       643

    accuracy                         0.7665      2244
   macro avg     0.7478    0.8001    0.7491      2244
weighted avg     0.8285    0.7665    0.7773      2244


              precision    recall  f1-score   support

          IS     0.9468    0.7461    0.8345      1646
          ES     0.5591    0.8848    0.6852       599

    accuracy                         0.7831      2245
   macro avg     0.7529    0.8154    0.7599      2245
weighted avg     0.8433    0.7831    0.7947      2245

              precision    recall  f1-score   support

          IS     0.9335    0.7392    0.8250      1614
          ES     0.5646    0.8653    0.6834       631

    accuracy                         0.7746      2245
   macro avg     0.7491    0.8022    0.7542      2245
weighted avg     0.8298    0.7746    0.78

In [ ]:
# 将结果转换为numpy数组
true_labels_all = np.array(true_labels_all)
predicted_labels_all = np.array(predicted_labels_all)

# 创建一个DataFrame
results_df = pd.DataFrame({
    'True_Label': true_labels_all,
    'Predicted_Label': predicted_labels_all
})

# 将DataFrame保存为CSV文件
results_df.to_csv('cnn_cross_validation.csv', index=False)